# Implementing CCSD with Pair Natural Orbitals

In [1]:
"""Tutorial implementing PNO-CCSD, simplified version of DLPNO-CCSD method"""

__authors__ = "A. Jiang and A. Wallace"
__credits__ = "A. Jiang and J. Madriaga"
__email__   = "ajiang10224@gmail.com"

__copyright__ = "(c) 2014-2023, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "05/15/2023"

## I. Theoretical Overview
In this tutorial, we are implementing PNO-CCSD, or Coupled-cluster theory with pair natural orbitals. Pair naturals reduce the size of the virtual space of each pair of occupied molecular orbitals by diagonalizing the pair density.
    
$$ \mathbf{D}_{ij} = \mathbf{\widetilde{t}}_{ij}\mathbf{t}_{ij}^{T} + \mathbf{\widetilde{t}}_{ij}^{T}\mathbf{t}_{ij}$$

where

$$ \mathbf{\widetilde{t}}_{ij} = 2\mathbf{t}_{ij} - \mathbf{t}_{ij}^{T} $$

The pair density is then diagonalized to form the PNO transformation matrix $\mathbf{X}^{PNO,ij}$, and the eigenvalues form the occupation numbers $\mathbf{n}^{occ,ij}$.

$$ \mathbf{D}_{ij}\mathbf{X}^{PNO,ij} = \mathbf{X}^{PNO,ij}\mathbf{n}^{occ,ij} $$

The columns of the $\mathbf{X}$ matrix are then truncated if the occupation number falls below a certain tolerance. In larger molecules, the size of the PNO space of each pair of occupied molecular orbitals is constant.

The transformation from canonical MOs to PNOs of a pair $ij$ is given by this formula:

$$ |\Phi_{a_{ij}}> = X^{PNO,ij}_{aa_{ij}}|\Phi_{a}> $$

Applying PNOs to CCSD theory reduces its scaling from $\mathcal{O}(N^{6})$ to $\mathcal{O}(N^{4})$, though the MP2 step still takes $\mathcal{O}(N^{5})$. In the state of the art DLPNO (Domain-Localized Pair Natural Orbital) approaches to Coupled-Cluster methods, the PNOs are not expanded in terms of canonical virtual orbitals, but in terms of Projected Atomic Orbitals (PAOs). Also, the occupied molecular orbital space is localized through means such as Foster-Boys or Pipek-Mezey localization, and local density-fitting is applied to localize the set of auxiliary basis functions for each local MO pair $ij$. The combination of these techniques allow DLPNO-CCSD to approach linear-scaling with larger molecules. In this tutotial, for simplicity and pedagogical purposes, we will not make use of these techniques.

## II. Implementation
We will now write a program that runs the PNO-CCSD algorithm. First, we need to import Psi4 and NumPy.

In [2]:
import psi4
import itertools
import numpy as np

psi4.set_memory(int(2e9))
numpy_memory = 2
psi4.core.set_output_file('output.dat')


  Memory set to   1.863 GiB by Python driver.


In [3]:
# Set molecule and wavefunction, and run preceeding SCF computation
mol = psi4.geometry("""
0 1
O
H 1 0.96
H 1 0.96 2 104.5
symmetry c1
""")

psi4.set_options({
    'basis': 'cc-pVDZ',
    'scf_type': 'df',
    'mp2_type': 'df',
    'cc_type': 'df',
    'nat_orbs': False,
    'e_convergence': 1.0e-8,
    'd_convergence': 1.0e-8,
    'r_convergence': 1.0e-6
})

# Get the SCF energy and wavefunction
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)

# Number of occupied orbitals and MOs
ndocc = scf_wfn.nalpha()
nmo = scf_wfn.nmo()
nvirt = nmo - ndocc

# Get orbital energies, cast into NumPy array, and separate occupied & virtual
eps = np.asarray(scf_wfn.epsilon_a())
e_ij = eps[:ndocc]
e_ab = eps[ndocc:]

# Get MO coefficients
Cmo = np.asarray(scf_wfn.Ca())
Cocc = Cmo[:,:ndocc]
Cvirt = Cmo[:,ndocc:]

Next, we need to form the density-fitted ERIs in the MO basis

In [4]:
# ==> Density Fitted ERIs <==
# Build auxiliary basis set
aux = psi4.core.BasisSet.build(mol, "DF_BASIS_SCF", "", "RIFIT", "cc-pVDZ")

# Build instance of Mints object
orb = scf_wfn.basisset()
mints = psi4.core.MintsHelper(orb)

# Build a zero basis
zero_bas = psi4.core.BasisSet.zero_ao_basis_set()

# Raw 3-index
Ppq = np.squeeze(mints.ao_eri(aux, zero_bas, orb, orb))

# Build and invert the Coulomb metric
metric = mints.ao_eri(aux, zero_bas, aux, zero_bas)
metric.power(-0.5, 1.e-14)
metric = np.squeeze(metric)

Qpq = np.einsum("QP,Ppq->Qpq", metric, Ppq, optimize=True)

# Transform into MO basis
Qia = np.einsum('pi,Qpq,qa->Qia', Cocc, Qpq, Cvirt, optimize=True)

Next up, we compute the amplitudes from MP2, in order to form our PNOs (pair natural orbitals), we need to compute the amplitudes with each MO pair. We later use this information to compute the pair density

In [5]:
K_ijab = np.empty((ndocc, ndocc, nvirt, nvirt), dtype=np.float64) # (ia|jb) integrals
T_ijab = np.empty((ndocc, ndocc, nvirt, nvirt), dtype=np.float64) # amplitudes
Tt_ijab = np.empty((ndocc, ndocc, nvirt, nvirt), dtype=np.float64) # anti-symmetrized amplitudes

e_vv = e_ab.reshape(-1, 1) + e_ab

E_MP2 = 0.0
for i,j in itertools.product(range(ndocc), range(ndocc)):
    K_ijab[i,j] = np.einsum('Qa,Qb->ab', Qia[:,i,:], Qia[:,j,:], optimize=True)
    T_ijab[i,j] = K_ijab[i,j] / (e_ij[i] + e_ij[j] - e_vv)
    Tt_ijab[i,j] = 2.0 * T_ijab[i,j] - T_ijab[i, j].transpose()
    E_MP2 += np.dot(Tt_ijab[i,j].flatten(), K_ijab[i,j].flatten())

Next, we perform a sanity check, to ensure that our DF-MP2 energy matches the Psi4 DF-MP2 energy

In [6]:
E_MP2_PSI = psi4.energy('mp2')

print('Our MP2 Correlation Energy:', E_MP2)
print('PSI MP2 Correlation Energy:', E_MP2_PSI - scf_e)
assert(np.allclose(scf_e + E_MP2, E_MP2_PSI))

Our MP2 Correlation Energy: -0.2041247114211424
PSI MP2 Correlation Energy: -0.2041247114217839


Next, we compute and diagonalize our pair densities to get the transformation matrix from canonical MOs to PNOs, and then compare the size of virtual space before and after PNO transformation

In [7]:
D_ij = np.empty((ndocc, ndocc, nvirt, nvirt), dtype=np.float64) # Pair densities
X_pno = np.empty((ndocc, ndocc), dtype=np.ndarray) # PNO transformation matrix
n_pno = np.zeros((ndocc, ndocc), dtype=np.int32) # Number of Pair Natural Orbitals per MO pair ij
e_pno = np.empty((ndocc, ndocc), dtype=np.ndarray) # PNO orbital energies per MO pair ij

# Compute Fock matrix elements
F_ao = np.asarray(scf_wfn.Fa())

F_ij = np.einsum('pi,pq,qj->ij', Cocc, F_ao, Cocc, optimize=True)
F_ab = np.einsum('pa,pq,qb->ab', Cvirt, F_ao, Cvirt, optimize=True)
assert(np.allclose(np.diag(F_ij), e_ij))
assert(np.allclose(np.diag(F_ab), e_ab))

# Occupation Number to Truncate PNOs
T_CUT_PNO = 1.0e-8
# T_CUT_PNO = 0.0

E_LMP2 = 0.0

for i,j in itertools.product(range(ndocc), range(ndocc)):

    if i > j: continue

    D_ij[i,j] = Tt_ijab[i,j] @ T_ijab[i,j].transpose() + Tt_ijab[i,j].transpose() @ T_ijab[i,j]
    n_occ, X_pno[i,j] = np.linalg.eigh(D_ij[i,j])
    
    # Truncate PNOs based on occupation number
    for occ_num in n_occ:
        if (occ_num >= T_CUT_PNO):
            n_pno[i,j] += 1
    # n_pno[i,j] = 19
        
    X_pno[i,j] = X_pno[i,j][:,-n_pno[i,j]:]

    # Canonicalize PNOs
    e_pno_temp = np.linalg.multi_dot([X_pno[i,j].T, F_ab, X_pno[i,j]])
    e_pno_ij, pno_canon = np.linalg.eigh(e_pno_temp)
    # print(f"{e_pno_ij = } {i = } {j = }")
    X_pno[i,j] = X_pno[i,j] @ pno_canon

    #F_ab_canon = np.linalg.multi_dot([X_pno[i,j].T, F_ab, X_pno[i,j]])
    #F_ab_canon_off_diag = F_ab_canon - np.diag(np.diag(F_ab_canon))
    #print(np.sum(np.abs(F_ab_canon_off_diag), axis=None))

    K_ij = X_pno[i,j].T @ K_ijab[i,j] @ X_pno[i,j]
    T_ij = K_ij.copy()
    for a, b in itertools.product(range(n_pno[i,j]), range(n_pno[i,j])):
        T_ij[a,b] = T_ij[a,b] / (-e_pno_ij[a] - e_pno_ij[b] + F_ij[i,i] + F_ij[j,j])
    
    E_LMP2 += np.dot((2.0 * T_ij - T_ij.T).flatten(), K_ij.flatten())
    
    e_pno[i,j] = e_pno_ij

for i,j in itertools.product(range(ndocc), range(ndocc)):
    if i < j:
        e_pno[j,i] = e_pno[i,j].copy()
        X_pno[j,i] = X_pno[i,j].copy()
        n_pno[j,i] = n_pno[i,j].copy()

print(f'{E_LMP2 = }')
# Compare virtual space sizes with and without PNOs
print('Size of Non-Truncated Virtual Space', nvirt)
print('Average Size of Truncated PNO Virtual Space', int(np.round(np.mean(n_pno))))

E_LMP2 = -0.13376980554686912
Size of Non-Truncated Virtual Space 19
Average Size of Truncated PNO Virtual Space 14


The size reduction is not great for a small molecule like water, but the difference between the canonical MO size and the PNO size becomes greatly exaggerated for larger systems. Next, we need to recompute the integrals in the PNO basis

In [8]:
Qij = np.einsum('pi,Qpq,qj->Qij', Cocc, Qpq, Cocc, optimize=True)
Qia = np.einsum('pi,Qpq,qa->Qia', Cocc, Qpq, Cvirt, optimize=True)
Qab = np.einsum('pa,Qpq,qb->Qab', Cvirt, Qpq, Cvirt, optimize=True)

# => Compute all the integral classes in Jiang 2023 <= #

K_occ = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 50

K_bar = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 51
L_bar = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 52

J_ij = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 53
K_ij = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 54
L_ij = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 55
M_ij = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 56

K_tilde = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 57
L_tilde = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 58

Qab_ij = np.empty((ndocc, ndocc), dtype=np.ndarray) # Jiang Eq. 59

for i,j in itertools.product(range(ndocc), range(ndocc)):
    K_occ[i,j] = np.einsum('Qm,Qn->mn', Qij[:,i,:], Qij[:,j,:], optimize=True)
    
    K_bar[i,j] = np.einsum('Qm,Qe->me', Qij[:,i,:], Qia[:,j,:], optimize=True)
    K_bar[i,j] = np.einsum('mE,Ee->me', K_bar[i,j], X_pno[i,j], optimize=True)
    
    J_ij[i,j] = np.einsum('Q,Qab->ab', Qij[:,i,j], Qab, optimize=True)
    J_ij[i,j] = np.einsum('Aa,AB,Bb->ab', X_pno[i,j], J_ij[i,j], X_pno[i,j], optimize=True)
    
    K_ij[i,j] = np.einsum('Qa,Qb->ab', Qia[:,i,:], Qia[:,j,:], optimize=True)
    K_ij[i,j] = np.einsum('Aa,AB,Bb->ab', X_pno[i,j], K_ij[i,j], X_pno[i,j], optimize=True)
    #K_ij[i,j] = X_pno[i,j].T @ K_ijab[i,j] @ X_pno[i,j]

    K_tilde[i,j] = np.einsum('Qe,Qaf->eaf', Qia[:,i,:], Qab, optimize=True)
    K_tilde[i,j] = np.einsum('EAF,Ee,Aa,Ff->eaf', K_tilde[i,j], X_pno[i,j], X_pno[i,j], X_pno[i,j], optimize=True)
    
    Qab_ij[i,j] = np.einsum('QAB,Aa,Bb->Qab', Qab, X_pno[i,j], X_pno[i,j], optimize=True)
    
for i,j in itertools.product(range(ndocc), range(ndocc)):
    L_bar[i,j] = 2.0 * K_bar[i,j] - K_bar[j,i]
    L_ij[i,j] = 2.0 * K_ij[i,j] - K_ij[i,j].transpose()
    M_ij[i,j] = 2.0 * K_ij[i,j] - J_ij[i,j]
    L_tilde[i,j] = 2.0 * K_tilde[i,j] - K_tilde[i,j].transpose(2, 1, 0)

Now that we have our integrals, we still need to compute overlaps between PNOs of different pairs. This connects
the virtual spaces of different pairs in the CCSD equations. Fock matrix elements are also computed for MOs and PNOs.

In [9]:
# Compute overlap matrix elements
S_ao = np.asarray(mints.ao_overlap())
S_mo = np.einsum('pa,pq,qb->ab', Cvirt, S_ao, Cvirt, optimize=True)

S_pno = np.empty((ndocc, ndocc, ndocc, ndocc), dtype=np.ndarray)
for i,j,k,l in itertools.product(range(ndocc), range(ndocc), range(ndocc), range(ndocc)):
    S_pno[i,j,k,l] = np.einsum('Aa,AB,Bb->ab', X_pno[i,j], S_mo, X_pno[k,l], optimize=True)

After all of this, we are ready for our CCSD iterations. First, let us define our one-particle intermediates (Jiang Equations 60-62)

In [10]:
# Jiang Equation 60
def compute_Fmi(T_ia, tau_tilde):
    Fmi = F_ij.copy()
    for m,n,i in itertools.product(range(ndocc), range(ndocc), range(ndocc)):
        T_n = S_pno[m,n,n,n] @ T_ia[n]
        Fmi[m,i] += np.dot(L_bar[m,n][i], T_n)
        
        L_temp = np.linalg.multi_dot([S_pno[i,n,m,n], L_ij[m,n], S_pno[m,n,i,n]])
        Fmi[m,i] += np.dot(L_temp.flatten(), tau_tilde[i,n].flatten())
        
    return Fmi

# Jiang Equation 61
def compute_Fbe(T_ia, tau_tilde):
    Fbe = np.empty((ndocc, ndocc), dtype=np.ndarray)
    for i, j in itertools.product(range(ndocc), range(ndocc)):
        # Fbe[i,j] = np.linalg.multi_dot([X_pno[i,j].T, F_ab, X_pno[i,j]])
        Fbe[i,j] = np.diag(e_pno[i,j]).copy()

        for m in range(ndocc):
            temp_a = np.einsum('fbe,F,Ff->be', L_tilde[m,j], T_ia[m], S_pno[m,m,m,j], optimize=True)
            Fbe[i,j] += np.linalg.multi_dot([S_pno[i,j,m,j], temp_a, S_pno[m,j,i,j]])
            for n in range(ndocc):
                temp_b = np.einsum('bf,ef->be', tau_tilde[m,n], L_ij[m,n], optimize=True)
                Fbe[i,j] -= np.linalg.multi_dot([S_pno[i,j,m,n], temp_b, S_pno[m,n,i,j]])

    return Fbe

# Jiang Equation 62
def compute_Fme(T_ia):
    Fme = np.empty((ndocc, ndocc), dtype=np.ndarray)
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        Fme[i,j] = np.empty((ndocc, n_pno[i,j]), dtype=np.float64)
        for m,n in itertools.product(range(ndocc), range(ndocc)):
            Fme[i,j][m] = np.einsum('eE,EF,Ff,f->e', S_pno[i,j,m,n], L_ij[m,n], S_pno[m,n,n,n], T_ia[n], optimize=True)

    return Fme

Next, let us define our two-particle intermediates (Jiang Equations 63-65)

In [11]:
# Jiang Equation 63
def compute_Wmnij(T_ia, tau):
    Wmnij = np.zeros((ndocc, ndocc, ndocc, ndocc), dtype=np.float64)
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        Wmnij[i,j] = K_occ[i,j].copy()
        for m,n in itertools.product(range(ndocc), range(ndocc)):
            Wmnij[i,j] += np.einsum('e,eE,E->', T_ia[j], S_pno[j,j,m,n], K_bar[m,n][i], optimize=True)
            Wmnij[i,j] += np.einsum('e,eE,E->', T_ia[i], S_pno[i,i,m,n], K_bar[n,m][j], optimize=True)
            K_temp = np.linalg.multi_dot([S_pno[i,j,m,n], K_ij[m,n], S_pno[m,n,i,j]])
            Wmnij[i,j] += np.dot(K_temp.flatten(), tau[i,j].flatten())

    return Wmnij

# Jiang Equation 64
def compute_Wbar(T_ia, tau_bar, T_ijab):
    Wbar = np.empty((ndocc, ndocc), dtype=np.ndarray)
    for m,j in itertools.product(range(ndocc), range(ndocc)):
        Wbar[m,j] = K_ij[j,m].copy()
        Wbar[m,j] += np.einsum('F,Ff,ebf->eb', T_ia[j], S_pno[j,j,m,j], K_tilde[m,j], optimize=True).transpose()
        for n in range(ndocc):
            Wbar[m,j] -= np.einsum('B,Bb,e->be', T_ia[n], S_pno[n,n,m,j], K_bar[j,m][n], optimize=True)
            Wbar[m,j] -= np.linalg.multi_dot([S_pno[m,j,m,n], K_ij[m,n], S_pno[m,n,j,n], tau_bar[j,n], S_pno[j,n,m,j]]).transpose()
            Wbar[m,j] += 0.5 * np.linalg.multi_dot([S_pno[m,j,m,n], L_ij[m,n], S_pno[m,n,j,n], T_ijab[n,j], S_pno[j,n,m,j]]).transpose()
    return Wbar

# Jiang Equation 65
def compute_Wtilde(T_ia, tau_bar):
    Wtilde = np.empty((ndocc, ndocc), dtype=np.ndarray)
    for m,j in itertools.product(range(ndocc), range(ndocc)):
        Wtilde[m,j] = -J_ij[m,j].copy()
        Wtilde[m,j] -= np.einsum('F,Ff,fbe->be', T_ia[j], S_pno[j,j,m,j], K_tilde[m,j], optimize=True)
        for n in range(ndocc):
            Wtilde[m,j] += np.einsum('B,Bb,E,Ee->be', T_ia[n], S_pno[n,n,m,j], K_bar[m,n][j], S_pno[m,n,m,j], optimize=True)
            Wtilde[m,j] += np.linalg.multi_dot([S_pno[m,j,j,n], tau_bar[j,n].T, S_pno[j,n,m,n], K_ij[m,n], S_pno[m,n,m,j]])

    return Wtilde

In [12]:
# Execute Equation 66
def compute_Ria(T_ia, Fbe, Fmi, Fme, Tt_ijab):
    R_ia = np.empty((ndocc), dtype=np.ndarray)
    for i in range(ndocc):
        R_ia[i] = np.einsum('ae,e->a', Fbe[i,i], T_ia[i], optimize=True)
        for m in range(ndocc):
            R_ia[i] += np.einsum("aA,AF,Ff,f->a", S_pno[i,i,i,m], M_ij[i,m], S_pno[i,m,m,m], T_ia[m], optimize=True)
            R_ia[i] -= Fmi[m,i] * np.einsum('A,Aa->a', T_ia[m], S_pno[m,m,i,i], optimize=True)
            R_ia[i] += np.einsum('aA,AE,E->a', S_pno[i,i,i,m], Tt_ijab[i,m], Fme[i,m][m], optimize=True)
            R_ia[i] += np.einsum('ef,eAf,Aa->a', Tt_ijab[m,i], K_tilde[m,i], S_pno[m,i,i,i], optimize=True)
            for n in range(ndocc):
                R_ia[i] -= np.einsum('aA,AE,E->a', S_pno[i,i,m,n], T_ijab[m,n], L_bar[m,n][i], optimize=True)
    
    return R_ia

# Jiang Equation 67
def compute_Rijab(T_ia, T_ijab, Tt_ijab, tau, tau_bar, tau_tilde, Fbe, Fmi, Fme, Wmnij, Wbar, Wtilde):
    R_bar_ijab = np.empty((ndocc, ndocc), dtype=np.ndarray)
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        R_bar_ijab[i,j] = 0.5 * K_ij[i,j].copy()
        R_bar_ijab[i,j] += np.einsum('ae,be->ab', T_ijab[i,j], Fbe[i,j], optimize=True)
        R_bar_ijab[i,j] += 0.5 * np.einsum('Qae,ef,Qbf->ab', Qab_ij[i,j], tau[i,j], Qab_ij[i,j])
        R_bar_ijab[i,j] += np.einsum('E,Ee,bea->ab', T_ia[i], S_pno[i,i,i,j], K_tilde[j,i], optimize=True)

        for m in range(ndocc):
            t_tilde = np.einsum('E,Ee->e', T_ia[m], S_pno[m,m,i,j], optimize=True)
            R_bar_ijab[i,j] -= 0.5 * np.einsum('ae,e,b->ab', T_ijab[i,j], Fme[i,j][m], t_tilde, optimize=True)
            R_bar_ijab[i,j] -= np.einsum('a,EBF,Ee,ef,fF,Bb->ab', t_tilde, K_tilde[m,j], S_pno[m,j,i,j], tau[i,j], S_pno[i,j,m,j], S_pno[m,j,i,j], optimize=True)
            R_bar_ijab[i,j] -= np.einsum('a,e,eE,EB,Bb->ab', t_tilde, T_ia[i], S_pno[i,i,m,j], K_ij[m,j], S_pno[m,j,i,j], optimize=True)
            R_bar_ijab[i,j] -= np.einsum('aA,AE,e,eE,b->ab', S_pno[i,j,m,j], J_ij[m,j], T_ia[i], S_pno[i,i,m,j], t_tilde, optimize=True)
            R_bar_ijab[i,j] -= np.einsum('a,b->ab', t_tilde, K_bar[i,j][m], optimize=True)
            R_bar_ijab[i,j] -= np.einsum('aA,AB,Bb->ab', S_pno[i,j,i,m], T_ijab[i,m], S_pno[i,m,i,j], optimize=True) * (Fmi[m,j] + 0.5 * np.einsum('E,E->', T_ia[j], Fme[j,j][m]))

            Wbar_ij = np.linalg.multi_dot([S_pno[i,j,m,j], Wbar[m,j], S_pno[m,j,i,m]])
            R_bar_ijab[i,j] += np.linalg.multi_dot([S_pno[i,j,i,m], (T_ijab[i,m] - T_ijab[i,m].transpose()), Wbar_ij.transpose()])
            Wsum_ij = np.linalg.multi_dot([S_pno[i,j,m,j], (Wbar[m,j] + Wtilde[m,j]), S_pno[m,j,i,m]])
            R_bar_ijab[i,j] += np.linalg.multi_dot([S_pno[i,j,i,m], T_ijab[i,m], Wsum_ij.transpose()])
            Wtilde_ij = np.linalg.multi_dot([S_pno[i,j,m,i], Wtilde[m,i], S_pno[m,i,m,j]])
            R_bar_ijab[i,j] += np.linalg.multi_dot([S_pno[i,j,m,j], T_ijab[m,j], Wtilde_ij.transpose()])

            for n in range(ndocc):
                R_bar_ijab[i,j] += 0.5 * np.einsum('aA,AB,Bb->ab', S_pno[i,j,m,n], tau[m,n], S_pno[m,n,i,j], optimize=True) * Wmnij[m,n,i,j]
    
    R_ijab = np.empty((ndocc, ndocc), dtype=np.ndarray)
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        R_ijab[i,j] = R_bar_ijab[i,j].copy() + R_bar_ijab[j,i].transpose() # Jiang Eq. 71
        #v = np.sum(np.square(np.subtract(R_ijab[i,j], K_ij[i,j])), axis=None)
    
    return R_ijab

In [13]:
# Initialize Residuals
R_ia = np.empty((ndocc), dtype=np.ndarray)
R_ijab = np.empty((ndocc, ndocc), dtype=np.ndarray)

# Initialize Amplitudes and intermediates
T_ia = np.empty((ndocc), dtype=np.ndarray)
T_ijab = np.empty((ndocc, ndocc), dtype=np.ndarray)
Tt_ijab = np.empty((ndocc, ndocc), dtype=np.ndarray)
tau = np.empty((ndocc, ndocc), dtype=np.ndarray)
tau_bar = np.empty((ndocc, ndocc), dtype=np.ndarray)
tau_tilde = np.empty((ndocc, ndocc), dtype=np.ndarray)

# Initialize to zeros
for i in range(ndocc):
    T_ia[i] = np.zeros((n_pno[i,i]), dtype=np.float64)
    R_ia[i] = np.zeros((n_pno[i,i]), dtype=np.float64)
    for j in range(ndocc):
        R_ijab[i,j] = np.zeros((n_pno[i,j], n_pno[i,j]), dtype=np.float64)
        T_ijab[i,j] = R_ijab[i,j].copy()
        Tt_ijab[i,j] = R_ijab[i,j].copy()
        tau[i,j] = R_ijab[i,j].copy()
        tau_bar[i,j] = R_ijab[i,j].copy()
        tau_tilde[i,j] = R_ijab[i,j].copy()

r_convergence = 1.0e-6
e_convergence = 1.0e-6

iter = 1
maxiter = 50
E_CCSD_old = 0.0

while (iter <= maxiter):
    # Compute One-Particle Intermediates
    Fmi = compute_Fmi(T_ia, tau_tilde)
    Fbe = compute_Fbe(T_ia, tau_tilde)
    Fme = compute_Fme(T_ia)

    # Compute Two-Particle Intermediates
    Wmnij = compute_Wmnij(T_ia, tau)
    Wbar = compute_Wbar(T_ia, tau_bar, T_ijab)
    Wtilde = compute_Wtilde(T_ia, tau_bar)

    # Compute Singles Residual
    R_ia = compute_Ria(T_ia, Fbe, Fmi, Fme, Tt_ijab)

    # Compute Doubles Residual
    R_ijab = compute_Rijab(T_ia, T_ijab, Tt_ijab, tau, tau_bar, tau_tilde, Fbe, Fmi, Fme, Wmnij, Wbar, Wtilde)

    # Update Singles and Doubles Residuals
    for i in range(ndocc):
        for a in range(n_pno[i,i]):
            T_ia[i][a] -= R_ia[i][a] / (e_pno[i,i][a] - F_ij[i,i])
    
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        for a, b in itertools.product(range(n_pno[i,j]), range(n_pno[i,j])):
            T_ijab[i,j][a,b] -= R_ijab[i,j][a,b] / (e_pno[i,j][a] + e_pno[i,j][b] - F_ij[i,i] - F_ij[j,j])

    # Update the intermediates (Jiang Eq. 68 - 70)
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        Tt_ijab[i,j] = 2.0 * T_ijab[i,j] - T_ijab[i,j].transpose()

        TiaTjb = np.einsum('aA,A,B,Bb->ab', S_pno[i,j,i,i], T_ia[i], T_ia[j], S_pno[j,j,i,j], optimize=True)
        tau[i,j] = T_ijab[i,j] + TiaTjb
        tau_bar[i,j] = 0.5 * T_ijab[i,j] + TiaTjb
        tau_tilde[i,j] = T_ijab[i,j] + 0.5 * TiaTjb

    # Compute Energy (finally), Jiang Eq 74
    E_CCSD = 0.0
    for i,j in itertools.product(range(ndocc), range(ndocc)):
        E_CCSD += np.einsum('ab,ab->', tau[i,j], L_ij[i,j])

    dE_CCSD = E_CCSD - E_CCSD_old

    print(f'@LCCSD Iter {iter}, E_CCSD: {E_CCSD:.12f}, dE_CCSD: {dE_CCSD:.12f}')

    E_CCSD_old = E_CCSD
    

    if (np.abs(dE_CCSD) < e_convergence):
        converged = True
        break

    iter += 1

E_CCSD_PSI = psi4.energy('ccsd')
print('E_CCSD PSI:', E_CCSD_PSI - scf_e)
print('E_PNO_CCSD:', E_CCSD)

@LCCSD Iter 1, E_CCSD: -0.204122014044, dE_CCSD: -0.204122014044
@LCCSD Iter 2, E_CCSD: -0.209989711422, dE_CCSD: -0.005867697379
@LCCSD Iter 3, E_CCSD: -0.213667822580, dE_CCSD: -0.003678111157
@LCCSD Iter 4, E_CCSD: -0.214624558226, dE_CCSD: -0.000956735647
@LCCSD Iter 5, E_CCSD: -0.215015445350, dE_CCSD: -0.000390887123
@LCCSD Iter 6, E_CCSD: -0.215169894401, dE_CCSD: -0.000154449051
@LCCSD Iter 7, E_CCSD: -0.215236356619, dE_CCSD: -0.000066462219
@LCCSD Iter 8, E_CCSD: -0.215265590912, dE_CCSD: -0.000029234293
@LCCSD Iter 9, E_CCSD: -0.215278940865, dE_CCSD: -0.000013349953
@LCCSD Iter 10, E_CCSD: -0.215285188693, dE_CCSD: -0.000006247828
@LCCSD Iter 11, E_CCSD: -0.215288192713, dE_CCSD: -0.000003004021
@LCCSD Iter 12, E_CCSD: -0.215289670880, dE_CCSD: -0.000001478166
@LCCSD Iter 13, E_CCSD: -0.215290414727, dE_CCSD: -0.000000743847
E_CCSD PSI: -0.21360221125424061
E_PNO_CCSD: -0.21529041472678492
